In [1]:
import cupy as cp
import pandas as pd
import seaborn as sns

Play on a grid, show in pygame

In [2]:
# Constants
global width
width = 1600
global height
height = 1000
global fps
fps =  30

# Transition probabilities - we should investigate parameters of this and relevant bifurcations
p_settle = 0.5 # Probability and grid member invades an empty square
p_rock_dominates = p_paper_dominates = p_scissors_dominates = 0.4 #Probability of colonizing neighbour





In [3]:
# build game
class RockPaperScissors():
  # build grid
  def __init__(self):
    self.grid = cp.zeros((width, height))

  #Assign rock->1, paper->2, scissors->3
  #Thus 3 > 2 > 1 > 3 > ...

  def seeding(self):
    ''' Get starting positions of our grid with density defined'''
    density = 0.15 #probability of seeding grid
    for x in range(width):
      for y in range(height):
        # Sample low-density choices for players
        value = cp.random.choice([1,2,3])
        toggle = cp.random.choice([True, False], p=[density, 1-density])
        self.grid[x][y] = value if toggle else 0


  def update(self):
    '''
    Markov process where we update our grid based on starting probabilities.
    Should do in place (updating self), and in parallel rather than in a loop
    '''
    new_grid = self.grid




To update the grid in parallel using CuPy, we can use array operations to identify neighbors and apply the transition rules. Here's an approach:

1. **Create a kernel**: Define a CuPy kernel that encapsulates the logic for updating a cell based on its neighbors and the given probabilities. This kernel will be applied to each cell in the grid in parallel.
2. **Apply the kernel**: Use `cupy.ElementwiseKernel` to apply the kernel to the grid. This allows CuPy to optimize the execution for parallelism.

In [7]:
# build game
class RockPaperScissors():
  # build grid
  def __init__(self):
    self.grid = cp.zeros((width, height))

  #Assign rock->1, paper->2, scissors->3
  #Thus 3 > 2 > 1 > 3 > ...

  def seeding(self):
    ''' Get starting positions of our grid '''
    density = 0.15 #probability of seeding grid
    for x in range(width):
      for y in range(height):
        # Sample low-density choices for players
        value = cp.random.choice([1,2,3],size=1)
        toggle = cp.random.choice([True, False], size=1, p=[density, 1-density])
        self.grid[x][y] = value if toggle else 0

  def update(self):
    '''
    Markov process where we update our grid based on starting probabilities.
    Should do in place (updating self), and in parallel rather than in a loop
    '''
    # Define the update kernel using cupy.ElementwiseKernel
    update_kernel = cp.ElementwiseKernel(
        'raw T grid, int width, int height, float p_settle, float p_rock_dominates, float p_paper_dominates, float p_scissors_dominates',
        'T new_grid',
        '''
        int x = i % width;
        int y = i / width;

        T current_cell = grid[i];
        new_grid = current_cell; // Initialize with the current value

        // Define neighbors' relative positions - bottom up each column
        int dx[] = {-1, -1, -1, 0, 0, 1, 1, 1};
        int dy[] = {-1, 0, 1, -1, 1, -1, 0, 1};

        // Count neighbors of each type
        int rock_neighbors = 0;
        int paper_neighbors = 0;
        int scissors_neighbors = 0;
        int empty_neighbors = 0;

        for (int j = 0; j < 8; ++j) {
            int nx = x + dx[j];
            int ny = y + dy[j];

            // Check boundaries
            if (nx >= 0 && nx < width && ny >= 0 && ny < height) {
                T neighbor_cell = grid[ny * width + nx]; // Access neighbor using linear index
                if (neighbor_cell == 1) {
                    rock_neighbors++;
                } else if (neighbor_cell == 2) {
                    paper_neighbors++;
                } else if (neighbor_cell == 3) {
                    scissors_neighbors++;
                } else {
                    empty_neighbors++;
                }
            }
        }

        // Apply update rules based on current cell and neighbors

        // If the current cell is empty (0)
        if (current_cell == 0) {
            // Settle with a random neighbor type with probability p_settle
            if (empty_neighbors < 8 && (float)cp::random::rand() < p_settle) {
                // Choose a random non-empty neighbor
                int neighbor_type = 0;
                while (neighbor_type == 0) {
                  int random_neighbor_index = (int)(cp::random::rand() * 8);
                   int nx = x + dx[random_neighbor_index];
                   int ny = y + dy[random_neighbor_index];
                   if (nx >= 0 && nx < width && ny >= 0 && ny < height) {
                     neighbor_type = grid[ny * width + nx];
                   }
                }
                 new_grid = neighbor_type;
            }
        } else { // If the current cell is not empty
            // Check for domination by neighbors

            // Rock (1) dominates Scissors (3)
            if (current_cell == 1 && scissors_neighbors > 0) {
                if ((float)cp::random::rand() < p_rock_dominates) {
                    new_grid = 3; // Becomes Scissors
                }
            }
            // Paper (2) dominates Rock (1)
            else if (current_cell == 2 && rock_neighbors > 0) {
                 if ((float)cp::random::rand() < p_paper_dominates) {
                    new_grid = 1; // Becomes Rock
                }
            }
            // Scissors (3) dominates Paper (2)
            else if (current_cell == 3 && paper_neighbors > 0) {
                 if ((float)cp::random::rand() < p_scissors_dominates) {
                    new_grid = 2; // Becomes Paper
                }
            }
        }

        ''',
        'update_kernel'
    )

    # Create a new grid to store the updated values
    new_grid = cp.copy(self.grid)

    # Apply the kernel to the grid
    update_kernel(self.grid, width, height, p_settle, p_rock_dominates, p_paper_dominates, p_scissors_dominates, new_grid)

    # Update the grid in place
    self.grid = new_grid

In [5]:
# Create grid and initialize. Visualize in pygame, with different colors


In [8]:
import pygame

# Initialize Pygame
pygame.init()

# Set up the display
screen = pygame.display.set_mode((width, height))
pygame.display.set_caption("Rock Paper Scissors Simulation")

# Define colors for each type
colors = {
    0: (0, 0, 0),      # Black for empty
    1: (255, 0, 0),    # Red for Rock
    2: (0, 255, 0),    # Green for Paper
    3: (0, 0, 255)     # Blue for Scissors
}

# Create an instance of the game
game = RockPaperScissors()
game.seeding()

# Game loop
running = True
clock = pygame.time.Clock()

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    # Update the game state (optional, you can update less frequently)
    # game.update()

    # Draw the grid
    for x in range(width):
        for y in range(height):
            cell_value = int(game.grid[x, y].get()) # Get value from CuPy array
            color = colors.get(cell_value, (255, 255, 255)) # Default to white if value not in colors
            screen.set_at((x, y), color)

    # Update the display
    pygame.display.flip()

    # Cap the frame rate
    clock.tick(fps)

pygame.quit()

KeyboardInterrupt: 